# Practical Lesson 9: Agricultural Production

### __Problem Description__:
It is necessary to plan agricultural production by allocating activities to three types of regions. The characteristics by region are:

|            |  A  |  B  |  C  |
|:----------:|:---:|:---:|:---:|
| Total area (ha) | 400 | 600 | 300 |
| Water avail. (m³) | 600 | 800 | 375 |

The products can be: wheat, cotton, and soybeans. The characteristics of the products are:

|            |  Wheat  | Cotton | Soybeans |
|:----------:|:-------:|:------:|:-------:|
| Max area (ha)  |   600   |  500   |  325   |
| Water cons. (m³/ha) |    3    |   2    |   1    |
| Profit ($/ha) |   400   |  300   |  100   |

Formulate a linear programming problem to allocate the activities and maximize profit.

In [1]:
from mip import *

def solve(model):
    model.verbose = 0
    status = model.optimize()

    print("Status = ", status)
    print(f"Solution value  = {model.objective_value}\n")

    print("Solution:")
    for v in model.vars:
      if v.x > 0.00001 and v.name.find("x")!=-1:
        print(v.name, " = ", v.x)

def save(model, filename):
    model.write(filename) 
    with open(filename, "r") as f: 
        print(f.read())

In [2]:
qtd_regions = 3
qtd_products = 3

# t_area[i] : total area (ha) of each region
t_area = [400, 600, 300]

# w_avail[i] : water availability (m³) in each region
w_avail = [600, 800, 375]

# mx_area[i] : maximum area (ha) for each product
mx_area = [600, 500, 325]

# w_cons[i] : water consumption (m³/ha) for each product
w_cons = [3, 2, 1]

# p[i] : profit ($/ha) for each product
p = [400, 300, 100]

In [9]:
model = Model(name="Agricultural Production", sense=MAXIMIZE, solver_name=CBC)

# x[i][j] : area (ha) of product i in region j
x = [[model.add_var(name=f"x_{i}_{j}", lb=0, ub=mx_area[i], var_type=CONTINUOUS) for j in range(qtd_regions)] for i in range(qtd_products)]

# Objective function
model.objective = xsum(p[i] * x[i][j] for i in range(qtd_products) for j in range(qtd_regions))

# Total area constraints
for j in range(qtd_regions):
    model.add_constr(lin_expr=xsum(x[i][j] for i in range(qtd_products)) <= t_area[j], name=f"total_area_constraint_{j}")

# Max area constraints
for i in range(qtd_products):
    model.add_constr(lin_expr=xsum(x[i][j] for j in range(qtd_regions)) <= mx_area[i], name=f"max_area_constraint_{i}")

# Water constraints
for j in range(qtd_regions):
    model.add_constr(lin_expr=xsum(w_cons[i] * x[i][j] for i in range(qtd_products)) <= w_avail[j], name=f"water_constraint_{j}")

save(model, "../data/lesson9.lp")

\Problem name: Agricultural Production

Minimize
OBJROW: -400 x_0_0 -400 x_0_1 -400 x_0_2 -300 x_1_0 -300 x_1_1 -300 x_1_2 -100 x_2_0 -100 x_2_1 -100 x_2_2
Subject To
total_area_constraint_0:  x_0_0 + x_1_0 + x_2_0 <= 400
total_area_constraint_1:  x_0_1 + x_1_1 + x_2_1 <= 600
total_area_constraint_2:  x_0_2 + x_1_2 + x_2_2 <= 300
max_area_constraint_0:  x_0_0 + x_0_1 + x_0_2 <= 600
max_area_constraint_1:  x_1_0 + x_1_1 + x_1_2 <= 500
max_area_constraint_2:  x_2_0 + x_2_1 + x_2_2 <= 325
water_constraint_0:  3 x_0_0 + 2 x_1_0 + x_2_0 <= 600
water_constraint_1:  3 x_0_1 + 2 x_1_1 + x_2_1 <= 800
water_constraint_2:  3 x_0_2 + 2 x_1_2 + x_2_2 <= 375
Bounds
 0 <= x_0_0 <= 600
 0 <= x_0_1 <= 600
 0 <= x_0_2 <= 600
 0 <= x_1_0 <= 500
 0 <= x_1_1 <= 500
 0 <= x_1_2 <= 500
 0 <= x_2_0 <= 325
 0 <= x_2_1 <= 325
 0 <= x_2_2 <= 325
End



In [10]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 253333.3333333333

Solution:
x_0_1  =  133.33333333333331
x_0_2  =  125.0
x_1_0  =  300.0
x_1_1  =  200.0
